In [3]:
import random
import numpy as np

# Arguments
REWARD = -1 # constant reward for non-terminal states
REWARD2 = -1
DISCOUNT = 0.99
MAX_ERROR = 10**(-3)

global counter
counter = 0

# Set up the initial environment
NUM_ACTIONS = 4
ACTIONS = [(1, 0), (0, -1), (-1, 0), (0, 1)] # Down, Left, Up, Right
NUM_ROW = 4
NUM_COL = 4
U = [[0, 0, 0, 10], [0, 0, 0, -10], [0, 0, 0, 0], [0, 0, 0, 0]]
policy = [[random.randint(0, 3) for j in range(NUM_COL)] for i in range(NUM_ROW)] # construct a random policy
Value = np.matrix(U)
# Visualization


def printEnvironment(arr, policy=False):
    res = ""
    for r in range(NUM_ROW):
        res += "|"
        for c in range(NUM_COL):
            if r == c == 1:
                val = "WALL"
            elif r <= 1 and c == 3:
                val = "+10" if r == 0 else "-10"
            else:
                val = ["Down", "Left", "Up", "Right"][arr[r][c]]
            res += " " + val[:5].ljust(5) + " |" # format
        res += "\n"
    print(res)



# Get the utility of the state reached by performing the given action from the given state
def getU(U, r, c, action):
    dr, dc = ACTIONS[action]
    newR, newC = r+dr, c+dc
    if newR < 0 or newC < 0 or newR >= NUM_ROW or newC >= NUM_COL or (newR == newC == 1): # collide with the boundary or the wall
        return U[r][c]
    else:
        return U[newR][newC]

# Calculate the utility of a state given an action
def calculateU(U, r, c, action):
    if r == 3 and c == 2:
       u = REWARD2
    else:
       u = REWARD
   # u += 0.1 * DISCOUNT * getU(U, r, c, (action-1)%4)
    u += 1 * DISCOUNT * getU(U, r, c, action)
   # u += 0.1 * DISCOUNT * getU(U, r, c, (action+1)%4)
    return u

# Perform some simplified value iteration steps to get an approximation of the utilities
def policyEvaluation(policy, U):
    global counter
    while True:
        nextU = [[0, 0, 0, 10], [0, 0, 0, -10], [0, 0, 0, 0], [0, 0, 0, 0]]
        error = 0
        for r in range(NUM_ROW):
            for c in range(NUM_COL):
                if (r <= 1 and c == 3) or (r == c == 1):
                    continue
                nextU[r][c] = calculateU(U, r, c, policy[r][c]) # simplified Bellman update
                error = max(error, abs(nextU[r][c]-U[r][c]))
        U = nextU
        counter += 1
        if error < MAX_ERROR * (1-DISCOUNT) / DISCOUNT:
            break
    return U

def policyIteration(policy, U):
    print("During the policy iteration:\n")
    while True:
        U = policyEvaluation(policy, U)
        unchanged = True
        for r in range(NUM_ROW):
            for c in range(NUM_COL):
                if (r <= 1 and c == 3) or (r == c == 1):
                    continue
                maxAction, maxU = None, -float("inf")
                for action in range(NUM_ACTIONS):
                    u = calculateU(U, r, c, action)
                    if u > maxU:
                        maxAction, maxU = action, u
                if maxU > calculateU(U, r, c, policy[r][c]):
                    policy[r][c] = maxAction # the action that maximizes the utility
                    unchanged = False
        if unchanged:
            break
        printEnvironment(policy)
        print (np.matrix(U),'\n')
        print (counter)
    return policy

# Print the initial environment
print("The initial random policy is:\n")
printEnvironment(policy)
print (np.matrix(U),'\n')
# Policy iteration
policy = policyIteration(policy, U)

# Print the optimal policy
print("The optimal policy is:\n")
printEnvironment(policy)

print(counter)

The initial random policy is:

| Up    | Up    | Right | +10   |
| Up    | WALL  | Down  | -10   |
| Up    | Left  | Down  | Right |
| Down  | Up    | Right | Down  |

[[  0   0   0  10]
 [  0   0   0 -10]
 [  0   0   0   0]
 [  0   0   0   0]] 

During the policy iteration:

| Up    | Right | Right | +10   |
| Up    | WALL  | Up    | -10   |
| Up    | Left  | Down  | Up    |
| Down  | Up    | Right | Down  |

[[-99.99900475 -99.99900475   8.9         10.        ]
 [-99.99900475   0.         -99.99900475 -10.        ]
 [-99.99900475 -99.99900475 -99.99900475 -99.99900475]
 [-99.99900475 -99.99900475 -99.99900475 -99.99900475]] 

1146
| Right | Right | Right | +10   |
| Up    | WALL  | Up    | -10   |
| Up    | Left  | Up    | Up    |
| Down  | Up    | Right | Up    |

[[-99.99902455   7.811        8.9         10.        ]
 [-99.99902455   0.           7.811      -10.        ]
 [-99.99902455 -99.99902455 -99.99902455 -10.9       ]
 [-99.99902455 -99.99902455 -99.99902455 -99.99902455]] 